# Lab 3: Handling Missing Values In your Data

Machine learning models cannot work with missing values in a dataset (`NaNs` or `'-'`). It is crucial to fix any missing values in your data. The following are 2 ways to deal with missing data:
- deleting rows that contain missing features;
- replacing missing features using proper techniques.

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

In [2]:
# loading the 'Lab3_df1.csv' csv data
df1 = pd.read_csv('Lab3_df1.csv')

In [3]:
# preparing data, drop unnecessary columns
df1 = df1.drop(columns=['Unnamed: 0', 'index'])
df1.head()

,Examination Date,Gender,Age,Mode,H,M,DD,Result
0,6/10/2024,male,72,d,NaN,5.058993,1.481877,Negative
1,2/20/2024,male,9,e,104.087633,6.531724,2.266884,Negative
2,6/27/2024,female,59,f,102.904315,6.273313,0.396333,Positive
3,9/22/2023,male,23,g,152.651655,7.333626,0.557534,Positive
4,7/21/2024,female,17,g,67.239054,3.642516,3.765706,Positive


# Lab Activity One: Handling Missing Data Without Time-Series Specific Methods

#### You will be dealing with `NaN` values for the ``H`` and ``Mode`` columns. You will go through several ways of handling the missing values. For each method, you will first create a temporary copy of the master dataframe (`df1`) to not alter the original dataset.

In [4]:
# Checking how many missing values are in the H column
df1["H"].isnull().sum()

9

##### [A] Drop all the rows which contain `NaN` values in the `'H'` column. Be sure to only drop the rows with missing values in the `'H'` column and no other column. Remember to use the `df_clean` dataframe.

In [8]:
df_clean = df1.copy()


#### Instead of just deleting the rows we can try to replace with a logical value. Generally, the value to replace with is up to the data analyst (you) to figure out. Having domain knowledge helps.

##### [A] Using `df_temp0` replace all the `NaN` values in the `'H'` column with zero ($0$).

In [10]:
df_temp0 = df1.copy()
df_temp0["H"]= df_temp0["H"].fillna(0)

In [12]:
df_temp0.head()

,Examination Date,Gender,Age,Mode,H,M,DD,Result
0,6/10/2024,male,72,d,0.000000,5.058993,1.481877,Negative
1,2/20/2024,male,9,e,104.087633,6.531724,2.266884,Negative
2,6/27/2024,female,59,f,102.904315,6.273313,0.396333,Positive
3,9/22/2023,male,23,g,152.651655,7.333626,0.557534,Positive
4,7/21/2024,female,17,g,67.239054,3.642516,3.765706,Positive


##### [A] Using `df_temp_mean` replace all the `NaN` values in the `'H'` column with the *mean* of the `'H'` column.

In [11]:
df_temp_mean = df1.copy()


##### [A] Using `df_temp_median` replace all the `NaN` values in the `'H'` column with the *median* of the `'H'` column.

In [ ]:
df_temp_median = df1.copy()


##### [A] Using `df_temp_mode` replace all the `NaN` values in the `'H'` column with the *mode* of the `'H'` column.
> Hint: Replace with the mode is trickier that the two previous methods.

In [13]:
df_temp_mode = df1.copy()
df_temp_mean["H"]= df_temp_mean["H"].replace(np.nan,df_temp_mean["H"].mean())

In [14]:
df_temp_mean.head()

,Examination Date,Gender,Age,Mode,H,M,DD,Result
0,6/10/2024,male,72,d,104.269683,5.058993,1.481877,Negative
1,2/20/2024,male,9,e,104.087633,6.531724,2.266884,Negative
2,6/27/2024,female,59,f,102.904315,6.273313,0.396333,Positive
3,9/22/2023,male,23,g,152.651655,7.333626,0.557534,Positive
4,7/21/2024,female,17,g,67.239054,3.642516,3.765706,Positive


#### Now, you will be dealing with missing values for the `'Mode'` column. This contains categorical values, which are harder to replace with a logical value, unlike numeric (mean, median, mode). Below are the recommended steps for this:
- Replace with the most recurring category (similar to mode).
- Create a new category called 'missing' and replace with it that.
- Replace with a category depending on another feature (requires further data analysis).
- Simply delete the rows with missing values.

Note: For our dataset, instead of containing `NaN`s the `'Mode'` column has `'_'` to indicate missing values.

In [15]:
df_temp = df1.copy()
df_temp['Mode'].unique()

array(['d', 'e', 'f', 'g', '_', 'c', 'a'], dtype=object)

##### [A] Using `df_temp`, replace the `'_'` values with the string `'missing'`.

In [16]:
df_temp['Mode'] = df_temp['Mode'].replace('_','missing')

In [17]:
df_temp['Mode'].unique()

array(['d', 'e', 'f', 'g', 'missing', 'c', 'a'], dtype=object)

# Lab Activity Two: Handling Missing Data With Time-Series Specific Methods

The three most popular methods for dealing with time-series data are listed below
- **Last Observation Carried Forward (LOCF):** Replacing the missing value with the value in the previous cell
- **Next Observation Carried Backward (NOCB):** Replacing the missing value with the value in the next cell
- **Linear interpolation:** Replacing missing values with estimates from previous values

##### Preparing the data

In [19]:
#Loading data
df2 = pd.read_csv('Lab3_df2.csv')

# Dropping unwated column
df2 = df2.drop(columns=['Unnamed: 0'])

# Change datatype of the date column
df2['Inspection Date'] = pd.to_datetime(df2['Inspection Date'])
df2.head()

# Here we are splitting our dataframe into 2, depend on if the site of device in is Clinic 1 or 2
df2_1 = df2[df2["Site Name"] == 'Clinic1'].copy()
df2_2 = df2[df2["Site Name"] == 'Clinic2'].copy()

df2_1.reset_index(inplace=True, drop=True)
df2_2.reset_index(inplace=True, drop=True)

In [20]:
df2_1.head()

,Inspection Date,T1,T2,Site Name
0,2023-09-04,0.079601,0.542921,Clinic1
1,2023-09-27,NaN,9.613118,Clinic1
2,2023-10-06,NaN,4.298943,Clinic1
3,2023-10-21,0.877617,9.240019,Clinic1
4,2023-11-26,NaN,6.659528,Clinic1


In [21]:
df2_2.head()

,Inspection Date,T1,T2,Site Name
0,2024-01-04,0.927976,5.473454,Clinic2
1,2024-01-28,0.905114,7.223218,Clinic2
2,2024-02-04,0.683924,5.047370,Clinic2
3,2024-02-27,0.622961,1.971804,Clinic2
4,2024-04-10,NaN,10.822079,Clinic2


##### Checking for missing values in the `'T1'` column

In [22]:
df2_1['T1'].isnull().sum()

4

In [23]:
df2_1['T1'].isnull().sum()

4

##### [A] An important first step when using time-series specific methods is to sort the dataset by time (date in our case). Sort both `df2_1` and `df2_2` by the `'Inspection Date'` column.

In [24]:
df2_1 = df2_1.sort_values(by='Inspection Date')
df2_2 = df2_2.sort_values(by='Inspection Date')
df2_1.head()

,Inspection Date,T1,T2,Site Name
0,2023-09-04,0.079601,0.542921,Clinic1
1,2023-09-27,NaN,9.613118,Clinic1
2,2023-10-06,NaN,4.298943,Clinic1
3,2023-10-21,0.877617,9.240019,Clinic1
4,2023-11-26,NaN,6.659528,Clinic1


In [25]:
df2_2.head()

,Inspection Date,T1,T2,Site Name
0,2024-01-04,0.927976,5.473454,Clinic2
1,2024-01-28,0.905114,7.223218,Clinic2
2,2024-02-04,0.683924,5.047370,Clinic2
3,2024-02-27,0.622961,1.971804,Clinic2
4,2024-04-10,NaN,10.822079,Clinic2


#### Last Observation Carried Forward (LOCF)

##### [A] using `df_temp1`, Use LOCF to fill the `NaN` values of the entire `df2_1 dataset`. Read Pandas [documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.fillna.html) to learn more about the `fillna` function.

In [26]:
df_temp1 = df2_1.copy()
df_temp1= df_temp1.ffill()

In [27]:
df_temp1.head()

,Inspection Date,T1,T2,Site Name
0,2023-09-04,0.079601,0.542921,Clinic1
1,2023-09-27,0.079601,9.613118,Clinic1
2,2023-10-06,0.079601,4.298943,Clinic1
3,2023-10-21,0.877617,9.240019,Clinic1
4,2023-11-26,0.877617,6.659528,Clinic1


#### Next Observation Carried Backward (NOCB)



##### [A] Using `df_temp2`, Use NOCB to fill the `NaN` values of the entire dataset.

In [28]:
df_temp2 = df2_2.copy()
df_temp2= df_temp2.bfill()

In [29]:
df_temp2.head()

,Inspection Date,T1,T2,Site Name
0,2024-01-04,0.927976,5.473454,Clinic2
1,2024-01-28,0.905114,7.223218,Clinic2
2,2024-02-04,0.683924,5.047370,Clinic2
3,2024-02-27,0.622961,1.971804,Clinic2
4,2024-04-10,0.498139,10.822079,Clinic2


#### Linear interpolation



##### [A] Using `df_temp3`, Use Interpolation to fill the NaN values of the `'T1'` column.
> Hint: Set the index of the dataframe to be the Inspection column and use the `.interpolate function(method='index)`. Reading [documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.interpolate.html) of the functions will help you do this.

In [30]:
df_temp3 = df2_1.copy()

df_temp3.set_index('Inspection Date')  # Set time column as index
df_temp3['T1'] = df_temp3['T1'].interpolate(method='index')  # Linear interpolation

In [31]:
df_temp3.head()

,Inspection Date,T1,T2,Site Name
0,2023-09-04,0.079601,0.542921,Clinic1
1,2023-09-27,0.345606,9.613118,Clinic1
2,2023-10-06,0.611612,4.298943,Clinic1
3,2023-10-21,0.877617,9.240019,Clinic1
4,2023-11-26,0.814721,6.659528,Clinic1


In [32]:
df_temp3.shape

(13, 4)